In [1]:
] status

Status `~/.julia/environments/v1.10/Project.toml`
  [336ed68f] CSV v0.10.14
  [a93c6f00] DataFrames v1.7.0
  [87dc4568] HiGHS v1.9.3
  [7073ff75] IJulia v1.25.0
  [4076af6c] JuMP v1.23.2
  [91a5bcdd] Plots v1.40.8
  [37e2e46d] LinearAlgebra


In [2]:
using JuMP
using HiGHS
using DataFrames

In [3]:
mod = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [4]:
@variable(mod,x1>=0)
@variable(mod,x2>=0)
@variable(mod,x3>=0)
@variable(mod,x4>=0)

x4

In [5]:
@objective(mod, Min, x1+3x2+5x3+2x4)

x1 + 3 x2 + 5 x3 + 2 x4

In [6]:
@constraint(mod, c1, 1x1+1x2+9x3+5x4>=7)
@constraint(mod, c2, 3x1+5x2+8x4>=3)
@constraint(mod, c3, 1x1+6x3+13x4>=5);

In [7]:
print(mod)

In [8]:
optimize!(mod)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 5e+00]
  Bound  [0e+00, 0e+00]
  RHS    [3e+00, 7e+00]
Presolving model
3 rows, 3 cols, 8 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 2(-2); elements 6(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(22) 0s
          1     2.8000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  2.8000000000e+00
HiGHS run time      :          0.00


In [9]:
solution_summary(mod,verbose=true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.80000e+00
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 2.80000e+00
  Primal solution :
    x1 : 0.00000e+00
    x2 : 0.00000e+00
    x3 : 0.00000e+00
    x4 : 1.40000e+00
  Dual solution :
    c1 : 4.00000e-01
    c2 : 0.00000e+00
    c3 : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.81606e-04
  Simplex iterations : 1
  Barrier iterations : 0
  Node count         : -1


In [10]:
report = lp_sensitivity_report(mod);

In [11]:
function variable_report(xi) 
    return (
        name = name(xi),
        lower_bound = has_lower_bound(xi) ? lower_bound(xi) : -Inf,
        value = value(xi),
        upper_bound = has_upper_bound(xi) ? upper_bound(xi) : Inf,
        reduced_cost = reduced_cost(xi),
        obj_coefficient = coefficient(objective_function(mod), xi),
        allowed_decrease = report[xi][1],
        allowed_increase = report[xi][2],
    )
end

variable_report (generic function with 1 method)

In [12]:
variable_df = DataFrames.DataFrame(variable_report(xi) for xi in all_variables(mod))

Row,name,lower_bound,value,upper_bound,reduced_cost,obj_coefficient,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,x1,0.0,0.0,Inf,0.6,1.0,-0.6,Inf
2,x2,0.0,0.0,Inf,2.6,3.0,-2.6,Inf
3,x3,0.0,0.0,Inf,1.4,5.0,-1.4,Inf
4,x4,0.0,1.4,Inf,0.0,2.0,-2.0,0.777778


In [13]:
function constraint_report(c::ConstraintRef) 
    return (
        name = name(c),
        value = value(c),
        rhs = normalized_rhs(c),
         slack = normalized_rhs(c) - value(c),
        shadow_price = shadow_price(c),
        allowed_decrease = report[c][1],
        allowed_increase = report[c][2],
    )
    end

constraint_report (generic function with 1 method)

In [14]:
constraint_df = DataFrames.DataFrame(constraint_report(ci) 
    for (F, S) in list_of_constraint_types(mod) 
        for ci in all_constraints(mod, F, S) 
            if F == AffExpr
    )

Row,name,value,rhs,slack,shadow_price,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,c1,7.0,7.0,0.0,-0.4,-5.07692,Inf
2,c2,11.2,3.0,-8.2,-0.0,-Inf,8.2
3,c3,18.2,5.0,-13.2,-0.0,-Inf,13.2
